# Interactive Interface

In addition to the command line tools, you can also work with an ActivitySim
model in an interactive environment, such as a Jupyter Notebook.  Much of the
interface for working in this way can be found in the `activitysim.core.workflow`
module.

In [ ]:
from activitysim.core import workflow
from activitysim import abm

To work with a built-in example model, you can use the
{py:func}`workflow.create_example <activitysim.core.workflow.create_example>`
function shortcut to get a model state to experiment with.

In [ ]:
state = workflow.create_example("prototype_mtc", directory="/tmp/examples")

To work with an existing model (not necessarily an example model), you can use
{py:meth}`State.make_default <activitysim.core.workflow.State.make_default>`

In [ ]:
state = workflow.State.make_default(
    working_dir="/tmp/examples/prototype_mtc",
    config_dirs="configs",
    data_dir="data",
    output_dir="output",
)

## Manipulating Settings

On major advantage of running ActivitySim in an interactive environment
is the ability to manipulate model features (e.g. settings, file locations) 
before executing the model.  For example, we can see and alter the number of 
households in our simulation sample.

In [ ]:
state.settings.households_sample_size

The settings files we loaded are configured to run a sample of 100 thousand
households.  To make that a smaller (or larger) number, we can just assign to 
that setting:

In [ ]:
state.settings.households_sample_size = 5000
state.settings.households_sample_size

The :ref:`Settings.households_sample_size` setting is required to be
an integer. If we give it a floating point number instead, that value
will be coerced to an integer (by dropping the fractional part).

In [ ]:
state.settings.households_sample_size = 3500.8
state.settings.households_sample_size

We can even give it a string that can be coerced to into an integer...

In [ ]:
state.settings.households_sample_size = "2500"

... but if it's a value that just simply can't be interpreted as an integer, you'll get an error.

In [ ]:
try:
    state.settings.households_sample_size = "banana"
except Exception as err:
    print(err)

## Model Execution

Once you are done editing any settings or configs, you can run the model, either all at once, or stepwise through individual components; the
{py:meth}`State.run <activitysim.core.workflow.State.run>`
accessor allows for either.  To run an individual component, you can 
call it as a method on {py:meth}`State.run <activitysim.core.workflow.State.run>`:

In [ ]:
state.run.initialize_landuse()

This way of running also offers tab-completion in compatible environments,
so you can type `state.run.ini<tab>` and see all the model components that
have names starting with "ini".

Alternatively, you can pass a list of model component names, to run multiple
components in a block together.

In [ ]:
state.run([
    "initialize_households", 
    "compute_accessibility", 
    "school_location", 
])

Note that when running interactively like this, it is up to the user to
monitor the correct sequence of components, as the list of components in
{py:attr}`Settings.models <activitysim.core.configuration.Settings.models>` 
is ignored in favor of the user's interactive 
instructions.  So it is up to the user to ensure that all necessary
antecedent data is available for each component, either by running the 
components in the correct order, or otherwise manually preparing the data
tables with the necessary values.

## Manipulating Data

In addition to manipulating settings, running interactively allows a user to 
read (and potentially write) to data tables at intermediate points within the model
stream.  For example, the 
{py:meth}`State.get_dataframe <activitysim.core.workflow.State.get_dataframe>`
method can give us access to data tables as a pandas DataFrame.

In [ ]:
accessibility_df = state.get_dataframe("accessibility")
accessibility_df.head()

We can use that DataFrame to do any Python analysis, summarization, 
or visualization that we like, using all the normal Python libraries
that can work with DataFrames.

In [ ]:
import altair as alt

alt.Chart(accessibility_df).mark_circle(size=100).encode(
    x=alt.X('auPkRetail:Q', scale=alt.Scale(zero=False)),
    y=alt.Y('auPkTotal:Q', scale=alt.Scale(zero=False)),
).interactive()

ActivitySim also includes a few convenient reporting functions, 
that can generate histograms or nominal distributions of data features.

In [ ]:
state.report.histogram(
    table_name="persons", 
    column_name="distance_to_school",
)

In [ ]:
state.report.nominal_distribution(
    tablename="persons", 
    nominal_col="ptype",
    row_grouping="female",
    categories={1: "1: Full-time worker",
                2: "2: Part-time worker",
                3: "3: College student",
                4: "4: Non-working adult",
                5: "5: Retired person",
                6: "6: Driving age student",
                7: "7: Non-driving student",
                8: "8: Pre-school child"}
)